# 1910456 Mir Shafayat Ahmed
## CSC470 Introduction To Parallel Programming
### Summer 2023 Assignment


<a href="https://colab.research.google.com/github/shafayat1004/Parallel-Programming-Assignments/blob/main/1910456_MirShafayatAhmed_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/shafayat1004/Parallel-Programming-Assignments.git
%pip install git+https://github.com/andreinechaev/nvcc4jupyter.git
!nvidia-smi

Cloning into 'Parallel-Programming-Assignments'...
remote: Enumerating objects: 158, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 158 (delta 15), reused 157 (delta 14), pack-reused 0
Receiving objects: 100% (158/158), 1.51 MiB | 8.18 MiB/s, done.
Resolving deltas: 100% (15/15), done.
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-bpxkduuj
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-bpxkduuj
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0a71d56e5dce3ff1f0dd2c47c29367629262f527
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4295 sha256=044a7262d9375a0821f480d98f542ba23a61acbeac9a97a12e9cb7d459515e1d
  Stored in directory: /tmp/pip-ephem-wheel-cache-uu2mqloe/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e

In [2]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [3]:
def build_and_run (assignment_no: int, no_of_test_cases: int):
    !nvcc mp{assignment_no}.cu -o mp{assignment_no}

    print(f"Running MP{assignment_no}")
    print("\n======================================================================")

    if no_of_test_cases > 0:
        for i in range(no_of_test_cases):
            print("----------------------------------------------------------------------")
            print(f"Test Case {i}:")
            print("----------------")
            !./mp{assignment_no} /content/Parallel-Programming-Assignments/MP{assignment_no}/data/{i}/*
    else:
        !./mp{assignment_no}

    print("======================================================================")

## MP0 (Test)

In [4]:
%%writefile mp0.cu
#include "/content/Parallel-Programming-Assignments/include/wb.h"

//@@ The purpose of this code is to become familiar with the submission
//@@ process. Do not worry if you do not understand all the details of
//@@ the code.

int main(int argc, char **argv) {
    int deviceCount;

    wbArg_read(argc, argv);

    cudaGetDeviceCount(&deviceCount);

    wbTime_start(GPU, "Getting GPU Data."); //@@ start a timer

    for (int dev = 0; dev < deviceCount; dev++) {
        cudaDeviceProp deviceProp;

        cudaGetDeviceProperties(&deviceProp, dev);

        if (dev == 0) {
            if (deviceProp.major == 9999 && deviceProp.minor == 9999) {
                wbLog(TRACE, "No CUDA GPU has been detected");
                return -1;
            } else if (deviceCount == 1) {
                //@@ WbLog is a provided logging API (similar to Log4J).
                //@@ The logging function wbLog takes a level which is either
                //@@ OFF, FATAL, ERROR, WARN, INFO, DEBUG, or TRACE and a
                //@@ message to be printed.
                wbLog(TRACE, "There is 1 device supporting CUDA");
            } else {
                wbLog(TRACE, "There are ", deviceCount,
                    " devices supporting CUDA");
            }
        }

        wbLog(TRACE, "Device ", dev, " name: ", deviceProp.name);
        wbLog(TRACE, " Computational Capabilities: ", deviceProp.major, ".",
            deviceProp.minor);
        wbLog(TRACE, " Maximum global memory size: ",
            deviceProp.totalGlobalMem);
        wbLog(TRACE, " Maximum constant memory size: ",
            deviceProp.totalConstMem);
        wbLog(TRACE, " Maximum shared memory size per block: ",
            deviceProp.sharedMemPerBlock);
        wbLog(TRACE, " Maximum block dimensions: ",
            deviceProp.maxThreadsDim[0], " x ", deviceProp.maxThreadsDim[1],
            " x ", deviceProp.maxThreadsDim[2]);
        wbLog(TRACE, " Maximum grid dimensions: ", deviceProp.maxGridSize[0],
            " x ", deviceProp.maxGridSize[1], " x ",
            deviceProp.maxGridSize[2]);
        wbLog(TRACE, " Warp size: ", deviceProp.warpSize);
    }

    wbTime_stop(GPU, "Getting GPU Data."); //@@ stop the timer

    return 0;
}

Writing mp0.cu


In [5]:
build_and_run(assignment_no = 0, no_of_test_cases = 0)

Running MP0

Trace main::30 There is 1 device supporting CUDA
Trace main::37 Device 0 name: Tesla T4
Trace main::38  Computational Capabilities: 7.5
Trace main::40  Maximum global memory size: 15835398144
Trace main::42  Maximum constant memory size: 65536
Trace main::44  Maximum shared memory size per block: 49152
Trace main::46  Maximum block dimensions: 1024 x 1024 x 64
Trace main::49  Maximum grid dimensions: 2147483647 x 65535 x 65535
Trace main::52  Warp size: 32
[GPU    ] 0.000294912 Getting GPU Data.


## MP1 (Vector Addition)

### Objective

The purpose of this lab is for you to become familiar with using the CUDA API by implementing a simple vector addition kernel
and its associated host code as shown in the lectures.

### Prerequisites

Before starting this lab, make sure that:

* You have completed all week 1 lectures or videos

* You have completed Lab0 (MP0)

* Chapter 2 of the text book would also be helpful

### Instruction

You should edit the code in `template.cu` to perform the following:

* Allocate device memory

* Copy host memory to device

* Initialize thread block and kernel grid dimensions

* Invoke CUDA kernel

* Copy results from device to host

* Free device memory

* Write the CUDA kernel

Instructions about where to place each part of the code is
demarcated by the `//@@` comment lines.

If your solution is correct, you should be able to see the following
output for each of the 10 test datasets:
```
--------------
Dataset  X
The input length is XX
Solution is correct
```




In [6]:
%%writefile mp1.cu
#include "/content/Parallel-Programming-Assignments/include/wb.h"

__global__ void vecAdd(float *in1, float *in2, float *out, int len) {
	int idx = blockIdx.x * blockDim.x + threadIdx.x;
	if (idx < len)
		out[idx] = in1[idx] + in2[idx];
}

int main(int argc, char **argv) {
	wbArg_t args;
	int inputLength;
	float *hostInput1;
	float *hostInput2;
	float *hostOutput;
	float *deviceInput1;
	float *deviceInput2;
	float *deviceOutput;

	args = wbArg_read(argc, argv);

	wbTime_start(Generic, "Importing data and creating memory on host");
	hostInput1 = (float *)wbImport(wbArg_getInputFile(args, 0), &inputLength);
	hostInput2 = (float *)wbImport(wbArg_getInputFile(args, 1), &inputLength);
	hostOutput = (float *)malloc(inputLength * sizeof(float));
	wbTime_stop(Generic, "Importing data and creating memory on host");

	wbLog(TRACE, "The input length is ", inputLength);

	int size = inputLength * sizeof(float);

	wbTime_start(GPU, "Allocating GPU memory.");
	cudaMalloc((void **)&deviceInput1, size);
	cudaMalloc((void **)&deviceInput2, size);
	cudaMalloc((void **)&deviceOutput, size);
	wbTime_stop(GPU, "Allocating GPU memory.");

	wbTime_start(GPU, "Copying input memory to the GPU.");
	cudaMemcpy(deviceInput1, hostInput1, size, cudaMemcpyHostToDevice);
	cudaMemcpy(deviceInput2, hostInput2, size, cudaMemcpyHostToDevice);
	wbTime_stop(GPU, "Copying input memory to the GPU.");

	dim3 DimGrid((inputLength - 1) / 256 + 1, 1, 1);
	dim3 DimBlock(256, 1, 1);

	wbTime_start(Compute, "Performing CUDA computation");
	vecAdd << <DimGrid, DimBlock >> >(deviceInput1, deviceInput2, deviceOutput, inputLength);

	cudaDeviceSynchronize();
	wbTime_stop(Compute, "Performing CUDA computation");

	wbTime_start(Copy, "Copying output memory to the CPU");
	cudaMemcpy(hostOutput, deviceOutput, size, cudaMemcpyDeviceToHost);
	wbTime_stop(Copy, "Copying output memory to the CPU");

	wbTime_start(GPU, "Freeing GPU Memory");
	cudaFree(deviceInput1);
	cudaFree(deviceInput2);
	cudaFree(deviceOutput);
	wbTime_stop(GPU, "Freeing GPU Memory");

	wbSolution(args, hostOutput, inputLength);

	free(hostInput1);
	free(hostInput2);
	free(hostOutput);

	return 0;
}


Writing mp1.cu


In [7]:
build_and_run(assignment_no = 1, no_of_test_cases = 10)

Running MP1

----------------------------------------------------------------------
Test Case 0:
----------------
[Generic] 0.000239104 Importing data and creating memory on host
Trace main::27 The input length is 64
[GPU    ] 0.000155136 Allocating GPU memory.
[GPU    ] 0.001193216 Copying input memory to the GPU.
[Compute] 0.000037120 Performing CUDA computation
[Copy   ] 0.000019968 Copying output memory to the CPU
[GPU    ] 0.000145920 Freeing GPU Memory
Solution is correct.
----------------------------------------------------------------------
Test Case 1:
----------------
[Generic] 0.000267776 Importing data and creating memory on host
Trace main::27 The input length is 128
[GPU    ] 0.000132096 Allocating GPU memory.
[GPU    ] 0.000041216 Copying input memory to the GPU.
[Compute] 0.000032768 Performing CUDA computation
[Copy   ] 0.000019968 Copying output memory to the CPU
[GPU    ] 0.000111872 Freeing GPU Memory
Solution is correct.
--------------------------------------------

## MP2 (Matrix Multiplication)

### Objective

The purpose of this lab is for you to practice with using the CUDA API by implementing a simple Matrix Multiply kernel and its associated host code as shown in the lectures.

### Prerequisites

Before starting this lab, make sure that:

* You have completed all week 2 lectures or videos

* You have completed Lab1 (MP1)

### Instruction

You should edit the code in `template.cu` to perform the following:

* Allocate device memory

* Copy host memory to device

* Initialize thread block and kernel grid dimensions

* Invoke CUDA kernel

* Copy results from device to host

* Free device memory

* Write the CUDA kernel

Instructions about where to place each part of the code is
demarcated by the `//@@` comment lines.

If your solution is correct, you should be able to see the
following output for each of the 10 test datasets:
```
--------------
Dataset  X
The dimensions of A are X x X
The dimensions of B are X x X
...
Solution is correct
```




In [8]:
  %%writefile mp2.cu
  #include "/content/Parallel-Programming-Assignments/include/wb.h"

  #define wbCheck(stmt)                                                     \
    do {                                                                    \
      cudaError_t err = stmt;                                               \
      if (err != cudaSuccess) {                                             \
        wbLog(ERROR, "Failed to run stmt ", #stmt);                         \
        wbLog(ERROR, "Got CUDA error ...  ", cudaGetErrorString(err));      \
        return -1;                                                          \
      }                                                                     \
    } while (0)

  __global__ void matrixMultiply(float *A, float *B, float *C, int numARows,
                                int numAColumns, int numBRows,
                                int numBColumns, int numCRows,
                                int numCColumns) {
    //@@ Insert code to implement matrix multiplication here
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    int bx = blockIdx.x;
    int by = blockIdx.y;
    int Row = by * blockDim.y + ty;
    int Col = bx * blockDim.x + tx;
    float Cvalue = 0.0;

    if (Row < numCRows && Col < numCColumns) {
      for (int k = 0; k < numAColumns; ++k) {
        Cvalue += A[Row * numAColumns + k] * B[k * numBColumns + Col];
      }
      C[Row * numCColumns + Col] = Cvalue;
    }
  }

  int main(int argc, char **argv) {
    wbArg_t args;
    float *hostA; // The A matrix
    float *hostB; // The B matrix
    float *hostC; // The output C matrix
    float *deviceA;
    float *deviceB;
    float *deviceC;
    int numARows;    // number of rows in the matrix A
    int numAColumns; // number of columns in the matrix A
    int numBRows;    // number of rows in the matrix B
    int numBColumns; // number of columns in the matrix B
    int numCRows;    // number of rows in the matrix C (you have to set this)
    int numCColumns; // number of columns in the matrix C (you have to set this)

    args = wbArg_read(argc, argv);

    wbTime_start(Generic, "Importing data and creating memory on host");
    hostA = (float *)wbImport(wbArg_getInputFile(args, 0), &numARows, &numAColumns);
    hostB = (float *)wbImport(wbArg_getInputFile(args, 1), &numBRows, &numBColumns);

    //@@ Set numCRows and numCColumns
    numCRows = numARows;
    numCColumns = numBColumns;

    //@@ Allocate the hostC matrix
    hostC = (float *)malloc(numCRows * numCColumns * sizeof(float));
    wbTime_stop(Generic, "Importing data and creating memory on host");

    wbLog(TRACE, "The dimensions of A are ", numARows, " x ", numAColumns);
    wbLog(TRACE, "The dimensions of B are ", numBRows, " x ", numBColumns);

    wbTime_start(GPU, "Allocating GPU memory.");
    //@@ Allocate GPU memory here
    cudaMalloc((void **)&deviceA, numARows * numAColumns * sizeof(float));
    cudaMalloc((void **)&deviceB, numBRows * numBColumns * sizeof(float));
    cudaMalloc((void **)&deviceC, numCRows * numCColumns * sizeof(float));
    wbTime_stop(GPU, "Allocating GPU memory.");

    wbTime_start(GPU, "Copying input memory to the GPU.");
    //@@ Copy memory to the GPU here
    cudaMemcpy(deviceA, hostA, numARows * numAColumns * sizeof(float), cudaMemcpyHostToDevice);
    cudaMemcpy(deviceB, hostB, numBRows * numBColumns * sizeof(float), cudaMemcpyHostToDevice);
    wbTime_stop(GPU, "Copying input memory to the GPU.");

    //@@ Initialize the grid and block dimensions here
    dim3 blockDim(16, 16);
    dim3 gridDim((numCColumns + blockDim.x - 1) / blockDim.x, (numCRows + blockDim.y - 1) / blockDim.y);

    wbTime_start(Compute, "Performing CUDA computation");
    //@@ Launch the GPU Kernel here
    matrixMultiply<<<gridDim, blockDim>>>(deviceA, deviceB, deviceC, numARows, numAColumns, numBRows, numBColumns, numCRows, numCColumns);
    cudaDeviceSynchronize();
    wbTime_stop(Compute, "Performing CUDA computation");

    wbTime_start(Copy, "Copying output memory to the CPU");
    //@@ Copy the GPU memory back to the CPU here
    cudaMemcpy(hostC, deviceC, numCRows * numCColumns * sizeof(float), cudaMemcpyDeviceToHost);
    wbTime_stop(Copy, "Copying output memory to the CPU");

    wbTime_start(GPU, "Freeing GPU Memory");
    //@@ Free the GPU memory here
    cudaFree(deviceA);
    cudaFree(deviceB);
    cudaFree(deviceC);
    wbTime_stop(GPU, "Freeing GPU Memory");

    wbSolution(args, hostC, numCRows, numCColumns);

    free(hostA);
    free(hostB);
    free(hostC);

    return 0;
  }


Writing mp2.cu


In [9]:
build_and_run(assignment_no = 2, no_of_test_cases = 10)

Running MP2

----------------------------------------------------------------------
Test Case 0:
----------------
[Generic] 0.003546880 Importing data and creating memory on host
Trace main::63 The dimensions of A are 64 x 64
Trace main::64 The dimensions of B are 64 x 64
[GPU    ] 0.000180992 Allocating GPU memory.
[GPU    ] 0.000057856 Copying input memory to the GPU.
[Compute] 0.000034816 Performing CUDA computation
[Copy   ] 0.000027904 Copying output memory to the CPU
[GPU    ] 0.000111872 Freeing GPU Memory
Solution is correct.
----------------------------------------------------------------------
Test Case 1:
----------------
[Generic] 0.007094016 Importing data and creating memory on host
Trace main::63 The dimensions of A are 128 x 64
Trace main::64 The dimensions of B are 64 x 128
[GPU    ] 0.000133120 Allocating GPU memory.
[GPU    ] 0.000068864 Copying input memory to the GPU.
[Compute] 0.000040960 Performing CUDA computation
[Copy   ] 0.000077824 Copying output memory to t

## MP3 (Tiled Matrix Multiplication)

### Objective

The purpose of this lab is for you to practice with using the CUDA API by implementing a tiled Matrix Multiply kernel and its associated host code as shown in the lectures.

### Prerequisites

Before starting this lab, make sure that:

* You have completed all week 2 lectures or videos

* You have completed Lab2 (MP2)

### Instruction

You should edit the code in `template.cu` to perform the following:

* Allocate device memory

* Copy host memory to device

* Initialize thread block and kernel grid dimensions

* Invoke CUDA kernel

* Copy results from device to host

* Free device memory

* Write the CUDA kernel

Instructions about where to place each part of the code is
demarcated by the `//@@` comment lines.

If your solution is correct, you should be able to see the
following output for each of the 10 test datasets:
```
--------------
Dataset  X
The dimensions of A are X x X
The dimensions of B are X x X
...
Solution is correct
```



In [10]:
%%writefile mp3.cu
#include "/content/Parallel-Programming-Assignments/include/wb.h"

#define wbCheck(stmt)                                                     \
  do {                                                                    \
    cudaError_t err = stmt;                                               \
    if (err != cudaSuccess) {                                             \
      wbLog(ERROR, "Failed to run stmt ", #stmt);                         \
      wbLog(ERROR, "Got CUDA error ...  ", cudaGetErrorString(err));      \
      return -1;                                                          \
    }                                                                     \
  } while (0)

#define TILE_SIZE 16

__global__ void matrixMultiply(float *A, float *B, float *C, int numARows,
                               int numAColumns, int numBRows,
                               int numBColumns, int numCRows,
                               int numCColumns) {

  __shared__ float tileA[TILE_SIZE][TILE_SIZE];
  __shared__ float tileB[TILE_SIZE][TILE_SIZE];

  int tx = threadIdx.x;
  int ty = threadIdx.y;
  int bx = blockIdx.x;
  int by = blockIdx.y;
  int Row = by * blockDim.y + ty;
  int Col = bx * blockDim.x + tx;
  float Cvalue = 0.0;


  for (int t = 0; t < (numAColumns - 1) / TILE_SIZE + 1; ++t) {

    if (Row < numARows && (t * TILE_SIZE + tx) < numAColumns) {
      tileA[ty][tx] = A[Row * numAColumns + t * TILE_SIZE + tx];

    } else {
      tileA[ty][tx] = 0.0;
    }

    if ((t * TILE_SIZE + ty) < numBRows && Col < numBColumns) {
      tileB[ty][tx] = B[(t * TILE_SIZE + ty) * numBColumns + Col];

    } else {
      tileB[ty][tx] = 0.0;
    }

    __syncthreads();


    for (int k = 0; k < TILE_SIZE; ++k) {
      Cvalue += tileA[ty][k] * tileB[k][tx];
    }

    __syncthreads();
  }


  if (Row < numCRows && Col < numCColumns) {
    C[Row * numCColumns + Col] = Cvalue;
  }
}

int main(int argc, char **argv) {
  wbArg_t args;
  float *hostA; // The A matrix
  float *hostB; // The B matrix
  float *hostC; // The output C matrix
  float *deviceA;
  float *deviceB;
  float *deviceC;
  int numARows;    // number of rows in the matrix A
  int numAColumns; // number of columns in the matrix A
  int numBRows;    // number of rows in the matrix B
  int numBColumns; // number of columns in the matrix B
  int numCRows;    // number of rows in the matrix C (you have to set this)
  int numCColumns; // number of columns in the matrix C (you have to set
                   // this)

  args = wbArg_read(argc, argv);

  wbTime_start(Generic, "Importing data and creating memory on host");
  hostA = (float *)wbImport(wbArg_getInputFile(args, 0), &numARows,
                            &numAColumns);
  hostB = (float *)wbImport(wbArg_getInputFile(args, 1), &numBRows,
                            &numBColumns);
  //@@ Set numCRows and numCColumns
  numCRows = numARows;
  numCColumns = numBColumns;
  //@@ Allocate the hostC matrix
  hostC = (float *)malloc(numCRows * numCColumns * sizeof(float));
  wbTime_stop(Generic, "Importing data and creating memory on host");

  wbLog(TRACE, "The dimensions of A are ", numARows, " x ", numAColumns);
  wbLog(TRACE, "The dimensions of B are ", numBRows, " x ", numBColumns);

  wbTime_start(GPU, "Allocating GPU memory.");
  //@@ Allocate GPU memory here
  cudaMalloc((void **)&deviceA, numARows * numAColumns * sizeof(float));
  cudaMalloc((void **)&deviceB, numBRows * numBColumns * sizeof(float));
  cudaMalloc((void **)&deviceC, numCRows * numCColumns * sizeof(float));
  wbTime_stop(GPU, "Allocating GPU memory.");

  wbTime_start(GPU, "Copying input memory to the GPU.");
  //@@ Copy memory to the GPU here
  cudaMemcpy(deviceA, hostA, numARows * numAColumns * sizeof(float), cudaMemcpyHostToDevice);
  cudaMemcpy(deviceB, hostB, numBRows * numBColumns * sizeof(float), cudaMemcpyHostToDevice);
  wbTime_stop(GPU, "Copying input memory to the GPU.");

  //@@ Initialize the grid and block dimensions here
  dim3 blockDim(TILE_SIZE, TILE_SIZE); // Use TILE_SIZE as the block size
  dim3 gridDim((numCColumns + TILE_SIZE - 1) / TILE_SIZE, (numCRows + TILE_SIZE - 1) / TILE_SIZE);

  wbTime_start(Compute, "Performing CUDA computation");
  //@@ Launch the GPU Kernel here
  matrixMultiply<<<gridDim, blockDim>>>(deviceA, deviceB, deviceC, numARows, numAColumns, numBRows, numBColumns, numCRows, numCColumns);
  cudaDeviceSynchronize();
  wbTime_stop(Compute, "Performing CUDA computation");

  wbTime_start(Copy, "Copying output memory to the CPU");
  //@@ Copy the GPU memory back to the CPU here
  cudaMemcpy(hostC, deviceC, numCRows * numCColumns * sizeof(float), cudaMemcpyDeviceToHost);
  wbTime_stop(Copy, "Copying output memory to the CPU");

  wbTime_start(GPU, "Freeing GPU Memory");
  //@@ Free the GPU memory here
  cudaFree(deviceA);
  cudaFree(deviceB);
  cudaFree(deviceC);
  wbTime_stop(GPU, "Freeing GPU Memory");

  wbSolution(args, hostC, numCRows, numCColumns);

  free(hostA);
  free(hostB);
  free(hostC);

  return 0;
}


Writing mp3.cu


In [11]:
build_and_run(assignment_no = 3, no_of_test_cases = 10)

Running MP3

----------------------------------------------------------------------
Test Case 0:
----------------
[Generic] 0.003866112 Importing data and creating memory on host
Trace main::94 The dimensions of A are 64 x 64
Trace main::95 The dimensions of B are 64 x 64
[GPU    ] 0.000178176 Allocating GPU memory.
[GPU    ] 0.000058112 Copying input memory to the GPU.
[Compute] 0.000041984 Performing CUDA computation
[Copy   ] 0.000027904 Copying output memory to the CPU
[GPU    ] 0.000118016 Freeing GPU Memory
Solution is correct.
----------------------------------------------------------------------
Test Case 1:
----------------
[Generic] 0.007862016 Importing data and creating memory on host
Trace main::94 The dimensions of A are 128 x 64
Trace main::95 The dimensions of B are 64 x 128
[GPU    ] 0.000143104 Allocating GPU memory.
[GPU    ] 0.000080896 Copying input memory to the GPU.
[Compute] 0.000040960 Performing CUDA computation
[Copy   ] 0.000089088 Copying output memory to t

## MP4

### Objective

The purpose of this lab is for you to practice with using the CUDA API by implementing a 3D Convolution kernel and its associated host code as shown in the lectures.

### Prerequisites

Before starting this lab, make sure that:

* You have completed all week 4 lectures or videos

* You have completed Lab3 (MP3)

### Instruction

You should edit the code in `template.cu` to perform the following:

* Allocate device memory

* Copy host memory to device

* Initialize thread block and kernel grid dimensions

* Invoke CUDA kernel

* Copy results from device to host

* Free device memory

* Write the CUDA kernel

If your solution is correct, you should be able to see the
following output for each of the 10 test datasets:
```
--------------
Dataset  X
The input size is XxYxZ
Solution is correct
```

